# Assignment 5

# Lab Assignment #5 – Using Trees and Priority Queues

## Exercise 1

**If your first name starts with a letter from A-J inclusively:**

Design the algorithm and method **following operations** for a binary
tree T:

- preorderNext(p): Return the position visited after p in a preorder
    traversal of T (or null if p is the last node visited).

- inorderNext(p): Return the position visited after p in an inorder
    traversal of T (or null if p is the last node visited).

Write a Java/Python to test your solution.

What are the worst-case running times of your algorithms?

The time complexity of the preorderNext method is O(log n) in the average case and O(n) in the worst case, where n is the number of nodes in the binary tree. This is because in the worst case scenario, the method may have to traverse up the entire height of the tree to find the next node. The space complexity of the method is O(1) because it only uses a constant amount of extra space regardless of the size of the input tree.

The time complexity of the inorderNext function is O(h), where h is the height of the tree. This is because in the worst case scenario, we may have to traverse all the way up to the root of the tree to find the next node in the inorder traversal. The space complexity of the inorderNext function is O(1) because we are not using any additional data structures that grow with the input size. We are simply using a constant amount of space for variables and pointers.

In [1]:
class Empty(Exception):
    pass


class LinkedQueue:
    class _Node:
        __slots__ = "_element", "_next"

        def __init__(self, element, next):
            self._element = element
            self._next = next

    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def first(self):
        if self.is_empty():
            raise Empty("Queue is empty")
        return self._head._element

    def dequeue(self):
        if self.is_empty():
            raise Empty("Queue is empty")
        answer = self._head._element
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():
            self._tail = None
        return answer

    def enqueue(self, e):
        newest = self._Node(e, None)
        if self.is_empty():
            self._head = newest
        else:
            self._tail._next = newest
        self._tail = newest
        self._size += 1


class Tree:
    class Position:
        def element(self):
            raise NotImplementedError("must be implemented by subclass")

        def __eq__(self, other):
            raise NotImplementedError("must be implemented by subclass")

        def __ne__(self, other):
            return not (self == other)

    def root(self):
        raise NotImplementedError("must be implemented by subclass")

    def parent(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def num_children(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def children(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def __len__(self):
        raise NotImplementedError("must be implemented by subclass")

    def is_root(self, p):
        return self.root() == p

    def is_leaf(self, p):
        return self.num_children(p) == 0

    def is_empty(self):
        return len(self) == 0

    def depth(self, p):
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))

    def _height1(self):
        return max(self.depth(p) for p in self.positions() if self.is_leaf(p))

    def _height2(self, p):
        if self.is_leaf(p):
            return 0
        else:
            return 1 + max(self._height2(c) for c in self.children(p))

    def height(self, p=None):
        if p is None:
            p = self.root()
        return self._height2(p)

    def __iter__(self):
        for p in self.positions():
            yield p.element()

    def positions(self):
        return self.preorder()

    def preorder(self):
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):
                yield p

    def _subtree_preorder(self, p):
        yield p
        for c in self.children(p):
            for other in self._subtree_preorder(c):
                yield other

    def postorder(self):
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p

    def _subtree_postorder(self, p):
        for c in self.children(p):
            for other in self._subtree_postorder(c):
                yield other
        yield p

    def breadthfirst(self):
        if not self.is_empty():
            fringe = LinkedQueue()
            fringe.enqueue(self.root())
            while not fringe.is_empty():
                p = fringe.dequeue()
                yield p
                for c in self.children(p):
                    fringe.enqueue(c)


class BinaryTree(Tree):
    def left(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def right(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def sibling(self, p):
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)

    def children(self, p):
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)

    def inorder(self):
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p

    def _subtree_inorder(self, p):
        if self.left(p) is not None:
            for other in self._subtree_inorder(self.left(p)):
                yield other
        yield p
        if self.right(p) is not None:
            for other in self._subtree_inorder(self.right(p)):
                yield other

    def positions(self):
        return self.inorder()


class LinkedBinaryTree(BinaryTree):
    class _Node:
        __slots__ = "_element", "_parent", "_left", "_right"

        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right

    class Position(BinaryTree.Position):
        def __init__(self, container, node):
            self._container = container
            self._node = node

        def element(self):
            return self._node._element

        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node

    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError("p must be proper Position type")
        if p._container is not self:
            raise ValueError("p does not belong to this container")
        if p._node._parent is p._node:
            raise ValueError("p is no longer valid")
        return p._node

    def _make_position(self, node):
        return self.Position(self, node) if node is not None else None

    def __init__(self):
        self._root = None
        self._size = 0

    def __len__(self):
        return self._size

    def root(self):
        return self._make_position(self._root)

    def parent(self, p):
        node = self._validate(p)
        return self._make_position(node._parent)

    def left(self, p):
        node = self._validate(p)
        return self._make_position(node._left)

    def right(self, p):
        node = self._validate(p)
        return self._make_position(node._right)

    def num_children(self, p):
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1
        if node._right is not None:
            count += 1
        return count

    def _add_root(self, e):
        if self._root is not None:
            raise ValueError("Root exists")
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)

    def _add_left(self, p, e):
        node = self._validate(p)
        if node._left is not None:
            raise ValueError("Left child exists")
        self._size += 1
        node._left = self._Node(e, node)
        return self._make_position(node._left)

    def _add_right(self, p, e):
        node = self._validate(p)
        if node._right is not None:
            raise ValueError("Right child exists")
        self._size += 1
        node._right = self._Node(e, node)
        return self._make_position(node._right)

    def _replace(self, p, e):
        node = self._validate(p)
        old = node._element
        node._element = e
        return old

    def _delete(self, p):
        node = self._validate(p)
        if self.num_children(p) == 2:
            raise ValueError("Position has two children")
        child = node._left if node._left else node._right
        if child is not None:
            child._parent = node._parent
        if node is self._root:
            self._root = child
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node
        return node._element

    def _attach(self, p, t1, t2):
        node = self._validate(p)
        if not self.is_leaf(p):
            raise ValueError("position must be leaf")
        if not type(self) is type(t1) is type(t2):
            raise TypeError("Tree types must match")
        self._size += len(t1) + len(t2)
        if not t1.is_empty():
            t1._root._parent = node
            node._left = t1._root
            t1._root = None
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2._root
            t2._root = None
            t2._size = 0

    def preorderNext(self, p):
        node = self._validate(p)
        # Case 1: If node has a left child, return the left child
        if node._left is not None:
            return self._make_position(node._left)
        # Case 2: If node has no left child but has a right child, return the right child
        if node._right is not None:
            return self._make_position(node._right)
        # Case 3: If node has neither left nor right children, traverse up the tree
        while node._parent is not None:
            if node == node._parent._left and node._parent._right is not None:
                return self._make_position(node._parent._right)
            node = node._parent
        return None

    def inorderNext(self, p):
        node = self._validate(p)
        # Case 1: If node has a right child, go to the leftmost node in the right subtree
        if node._right is not None:
            node = node._right
            while node._left is not None:
                node = node._left
            return self._make_position(node)
        # Case 2: If no right child, go up to the parent until we are a left child
        else:
            while node._parent is not None and node == node._parent._right:
                node = node._parent
            return self._make_position(node._parent)


def list_to_tree(values):
    tree = LinkedBinaryTree()
    if not values:
        return tree

    iter_values = iter(values)
    root_value = next(iter_values)
    if root_value is not None:
        root = tree._add_root(root_value)
        queue = LinkedQueue()
        queue.enqueue(root)

        while not queue.is_empty():
            node = queue.dequeue()

            try:
                left_value = next(iter_values)
                if left_value is not None:
                    left_node = tree._add_left(node, left_value)
                    queue.enqueue(left_node)
            except StopIteration:
                break

            try:
                right_value = next(iter_values)
                if right_value is not None:
                    right_node = tree._add_right(node, right_value)
                    queue.enqueue(right_node)
            except StopIteration:
                break

    return tree


# Create a binary tree from a list of values
tree = list_to_tree([1, 2, 3, 4, 5, 6, 7])

# Test preorderNext
print("Preorder Traversal:")
for position in tree.preorder():
    print(position.element(), end=" ")
print()

# Test preorderNext method
print("Preorder Next of each node:")
positions = list(tree.preorder())
for position in positions:
    next_pos = tree.preorderNext(position)
    if next_pos is not None:
        print(f"Node {position.element()} -> {next_pos.element()}")
    else:
        print(f"Node {position.element()} -> None")

# Test inorderNext
print("\nInorder Traversal:")
for position in tree.inorder():
    print(position.element(), end=" ")
print()

# Test inorderNext method
print("Inorder Next of each node:")
positions = list(tree.inorder())
for position in positions:
    next_pos = tree.inorderNext(position)
    if next_pos is not None:
        print(f"Node {position.element()} -> {next_pos.element()}")
    else:
        print(f"Node {position.element()} -> None")

Preorder Traversal:
1 2 4 5 3 6 7 
Preorder Next of each node:
Node 1 -> 2
Node 2 -> 4
Node 4 -> 5
Node 5 -> 3
Node 3 -> 6
Node 6 -> 7
Node 7 -> None

Inorder Traversal:
4 2 5 1 6 3 7 
Inorder Next of each node:
Node 4 -> 2
Node 2 -> 5
Node 5 -> 1
Node 1 -> 6
Node 6 -> 3
Node 3 -> 7
Node 7 -> None


**If your first name starts with a letter from K-Z inclusively:**

1. Design the algorithm and method **following operations** for a
    binary tree T:

- inorderNext(p): Return the position visited after p in an inorder
    traversal of T (or null if p is the last node visited).

- postorderNext(p): Return the position visited after p in a postorder
    traversal of T (or null if p is the last node visited).

Write a Java/Python to test your solution.

What are the worst-case running times of your algorithms?

(5 marks)

The time complexity of the inorderNext function is O(h), where h is the height of the tree. This is because in the worst case scenario, we may have to traverse all the way up to the root of the tree to find the next node in the inorder traversal. The space complexity of the inorderNext function is O(1) because we are not using any additional data structures that grow with the input size. We are simply using a constant amount of space for variables and pointers.

The time complexity of this postorderNext method is O(h), where h is the height of the tree. This is because in the worst case scenario, we may need to traverse from the current node to the root of the tree, which would involve visiting each node on the path from the current node to the root. The space complexity of this method is O(1) because we are not using any additional data structures that grow with the input size. We are simply using a constant amount of space to store variables and return the result.

In [1]:
class Empty(Exception):
    pass


class LinkedQueue:
    class _Node:
        __slots__ = "_element", "_next"

        def __init__(self, element, next):
            self._element = element
            self._next = next

    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def first(self):
        if self.is_empty():
            raise Empty("Queue is empty")
        return self._head._element

    def dequeue(self):
        if self.is_empty():
            raise Empty("Queue is empty")
        answer = self._head._element
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():
            self._tail = None
        return answer

    def enqueue(self, e):
        newest = self._Node(e, None)
        if self.is_empty():
            self._head = newest
        else:
            self._tail._next = newest
        self._tail = newest
        self._size += 1


class Tree:
    class Position:
        def element(self):
            raise NotImplementedError("must be implemented by subclass")

        def __eq__(self, other):
            raise NotImplementedError("must be implemented by subclass")

        def __ne__(self, other):
            return not (self == other)

    def root(self):
        raise NotImplementedError("must be implemented by subclass")

    def parent(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def num_children(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def children(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def __len__(self):
        raise NotImplementedError("must be implemented by subclass")

    def is_root(self, p):
        return self.root() == p

    def is_leaf(self, p):
        return self.num_children(p) == 0

    def is_empty(self):
        return len(self) == 0

    def depth(self, p):
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))

    def _height1(self):
        return max(self.depth(p) for p in self.positions() if self.is_leaf(p))

    def _height2(self, p):
        if self.is_leaf(p):
            return 0
        else:
            return 1 + max(self._height2(c) for c in self.children(p))

    def height(self, p=None):
        if p is None:
            p = self.root()
        return self._height2(p)

    def __iter__(self):
        for p in self.positions():
            yield p.element()

    def positions(self):
        return self.preorder()

    def preorder(self):
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):
                yield p

    def _subtree_preorder(self, p):
        yield p
        for c in self.children(p):
            for other in self._subtree_preorder(c):
                yield other

    def postorder(self):
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p

    def _subtree_postorder(self, p):
        for c in self.children(p):
            for other in self._subtree_postorder(c):
                yield other
        yield p

    def breadthfirst(self):
        if not self.is_empty():
            fringe = LinkedQueue()
            fringe.enqueue(self.root())
            while not fringe.is_empty():
                p = fringe.dequeue()
                yield p
                for c in self.children(p):
                    fringe.enqueue(c)


class BinaryTree(Tree):
    def left(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def right(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def sibling(self, p):
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)

    def children(self, p):
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)

    def inorder(self):
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p

    def _subtree_inorder(self, p):
        if self.left(p) is not None:
            for other in self._subtree_inorder(self.left(p)):
                yield other
        yield p
        if self.right(p) is not None:
            for other in self._subtree_inorder(self.right(p)):
                yield other

    def positions(self):
        return self.inorder()


class LinkedBinaryTree(BinaryTree):
    class _Node:
        __slots__ = "_element", "_parent", "_left", "_right"

        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right

    class Position(BinaryTree.Position):
        def __init__(self, container, node):
            self._container = container
            self._node = node

        def element(self):
            return self._node._element

        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node

    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError("p must be proper Position type")
        if p._container is not self:
            raise ValueError("p does not belong to this container")
        if p._node._parent is p._node:
            raise ValueError("p is no longer valid")
        return p._node

    def _make_position(self, node):
        return self.Position(self, node) if node is not None else None

    def __init__(self):
        self._root = None
        self._size = 0

    def __len__(self):
        return self._size

    def root(self):
        return self._make_position(self._root)

    def parent(self, p):
        node = self._validate(p)
        return self._make_position(node._parent)

    def left(self, p):
        node = self._validate(p)
        return self._make_position(node._left)

    def right(self, p):
        node = self._validate(p)
        return self._make_position(node._right)

    def num_children(self, p):
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1
        if node._right is not None:
            count += 1
        return count

    def _add_root(self, e):
        if self._root is not None:
            raise ValueError("Root exists")
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)

    def _add_left(self, p, e):
        node = self._validate(p)
        if node._left is not None:
            raise ValueError("Left child exists")
        self._size += 1
        node._left = self._Node(e, node)
        return self._make_position(node._left)

    def _add_right(self, p, e):
        node = self._validate(p)
        if node._right is not None:
            raise ValueError("Right child exists")
        self._size += 1
        node._right = self._Node(e, node)
        return self._make_position(node._right)

    def _replace(self, p, e):
        node = self._validate(p)
        old = node._element
        node._element = e
        return old

    def _delete(self, p):
        node = self._validate(p)
        if self.num_children(p) == 2:
            raise ValueError("Position has two children")
        child = node._left if node._left else node._right
        if child is not None:
            child._parent = node._parent
        if node is self._root:
            self._root = child
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node
        return node._element

    def _attach(self, p, t1, t2):
        node = self._validate(p)
        if not self.is_leaf(p):
            raise ValueError("position must be leaf")
        if not type(self) is type(t1) is type(t2):
            raise TypeError("Tree types must match")
        self._size += len(t1) + len(t2)
        if not t1.is_empty():
            t1._root._parent = node
            node._left = t1._root
            t1._root = None
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2._root
            t2._root = None
            t2._size = 0

    def inorderNext(self, p):
        node = self._validate(p)
        # Case 1: If node has a right child, go to the leftmost node in the right subtree
        if node._right is not None:
            node = node._right
            while node._left is not None:
                node = node._left
            return self._make_position(node)
        # Case 2: If no right child, go up to the parent until we are a left child
        else:
            while node._parent is not None and node == node._parent._right:
                node = node._parent
            return self._make_position(node._parent)

    def postorderNext(self, p):
        node = self._validate(p)
        parent = node._parent
        # Case 1: If node is root, return None
        if parent is None:
            return None
        # Case 2: If node is the left child and parent has a right child,
        # the next node in postorder is the leftmost node in the parent's right subtree
        if parent._right is not None and node == parent._left:
            node = parent._right
            while node._left is not None or node._right is not None:
                if node._left is not None:
                    node = node._left
                else:
                    node = node._right
            return self._make_position(node)
        # Case 3: Otherwise, the next node is the parent
        return self._make_position(parent)


def list_to_tree(values):
    tree = LinkedBinaryTree()
    if not values:
        return tree

    iter_values = iter(values)
    root_value = next(iter_values)
    if root_value is not None:
        root = tree._add_root(root_value)
        queue = LinkedQueue()
        queue.enqueue(root)

        while not queue.is_empty():
            node = queue.dequeue()

            try:
                left_value = next(iter_values)
                if left_value is not None:
                    left_node = tree._add_left(node, left_value)
                    queue.enqueue(left_node)
            except StopIteration:
                break

            try:
                right_value = next(iter_values)
                if right_value is not None:
                    right_node = tree._add_right(node, right_value)
                    queue.enqueue(right_node)
            except StopIteration:
                break

    return tree


# Create a binary tree from a list of values
tree = list_to_tree([1, 2, 3, 4, 5, 6, 7])

# Test inorderNext
print("Inorder Traversal:")
for position in tree.inorder():
    print(position.element(), end=" ")
print("\n")

# Test inorderNext method
print("Inorder Next of each node:")
positions = list(tree.inorder())
for position in positions:
    next_pos = tree.inorderNext(position)
    if next_pos is not None:
        print(f"Node {position.element()} -> {next_pos.element()}")
    else:
        print(f"Node {position.element()} -> None")

print("\nPostorder Traversal:")
for position in tree.postorder():
    print(position.element(), end=" ")
print("\n")

# Test postorderNext method
print("Postorder Next of each node:")
positions = list(tree.postorder())
for position in positions:
    next_pos = tree.postorderNext(position)
    if next_pos is not None:
        print(f"Node {position.element()} -> {next_pos.element()}")
    else:
        print(f"Node {position.element()} -> None")

Inorder Traversal:
4 2 5 1 6 3 7 

Inorder Next of each node:
Node 4 -> 2
Node 2 -> 5
Node 5 -> 1
Node 1 -> 6
Node 6 -> 3
Node 3 -> 7
Node 7 -> None

Postorder Traversal:
4 5 2 6 7 3 1 

Postorder Next of each node:
Node 4 -> 5
Node 5 -> 2
Node 2 -> 6
Node 6 -> 7
Node 7 -> 3
Node 3 -> 1
Node 1 -> None


## Exercise 2

Give an efficient algorithm that computes and prints, for every position
p of a tree T, the element of p followed by the height of p’s subtree.
Write a Java/Python to test your solution.

**Hint**: Use a postorder traversal to find the height of each subtree.
The height for a subtree at p will be 0 if p is a leaf and otherwise one
more than the height of the max child. Print out the element at p and
its computed height during the postorder visit.

(3 marks)

In [12]:
class Empty(Exception):
    pass


class LinkedQueue:
    class _Node:
        __slots__ = "_element", "_next"

        def __init__(self, element, next):
            self._element = element
            self._next = next

    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def first(self):
        if self.is_empty():
            raise Empty("Queue is empty")
        return self._head._element

    def dequeue(self):
        if self.is_empty():
            raise Empty("Queue is empty")
        answer = self._head._element
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():
            self._tail = None
        return answer

    def enqueue(self, e):
        newest = self._Node(e, None)
        if self.is_empty():
            self._head = newest
        else:
            self._tail._next = newest
        self._tail = newest
        self._size += 1


class Tree:
    class Position:
        def element(self):
            raise NotImplementedError("must be implemented by subclass")

        def __eq__(self, other):
            raise NotImplementedError("must be implemented by subclass")

        def __ne__(self, other):
            return not (self == other)

    def root(self):
        raise NotImplementedError("must be implemented by subclass")

    def parent(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def num_children(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def children(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def __len__(self):
        raise NotImplementedError("must be implemented by subclass")

    def is_root(self, p):
        return self.root() == p

    def is_leaf(self, p):
        return self.num_children(p) == 0

    def is_empty(self):
        return len(self) == 0

    def depth(self, p):
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))

    def _height1(self):
        return max(self.depth(p) for p in self.positions() if self.is_leaf(p))

    def _height2(self, p):
        if self.is_leaf(p):
            return 0
        else:
            return 1 + max(self._height2(c) for c in self.children(p))

    def height(self, p=None):
        if p is None:
            p = self.root()
        return self._height2(p)

    def __iter__(self):
        for p in self.positions():
            yield p.element()

    def positions(self):
        return self.preorder()

    def preorder(self):
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):
                yield p

    def _subtree_preorder(self, p):
        yield p
        for c in self.children(p):
            for other in self._subtree_preorder(c):
                yield other

    def postorder(self):
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p

    def _subtree_postorder(self, p):
        for c in self.children(p):
            for other in self._subtree_postorder(c):
                yield other
        yield p

    def breadthfirst(self):
        if not self.is_empty():
            fringe = LinkedQueue()
            fringe.enqueue(self.root())
            while not fringe.is_empty():
                p = fringe.dequeue()
                yield p
                for c in self.children(p):
                    fringe.enqueue(c)


class BinaryTree(Tree):
    def left(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def right(self, p):
        raise NotImplementedError("must be implemented by subclass")

    def sibling(self, p):
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)

    def children(self, p):
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)

    def inorder(self):
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p

    def _subtree_inorder(self, p):
        if self.left(p) is not None:
            for other in self._subtree_inorder(self.left(p)):
                yield other
        yield p
        if self.right(p) is not None:
            for other in self._subtree_inorder(self.right(p)):
                yield other

    def positions(self):
        return self.inorder()


class LinkedBinaryTree(BinaryTree):
    class _Node:
        __slots__ = "_element", "_parent", "_left", "_right"

        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right

    class Position(BinaryTree.Position):
        def __init__(self, container, node):
            self._container = container
            self._node = node

        def element(self):
            return self._node._element

        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node

    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError("p must be proper Position type")
        if p._container is not self:
            raise ValueError("p does not belong to this container")
        if p._node._parent is p._node:
            raise ValueError("p is no longer valid")
        return p._node

    def _make_position(self, node):
        return self.Position(self, node) if node is not None else None

    def __init__(self):
        self._root = None
        self._size = 0

    def __len__(self):
        return self._size

    def root(self):
        return self._make_position(self._root)

    def parent(self, p):
        node = self._validate(p)
        return self._make_position(node._parent)

    def left(self, p):
        node = self._validate(p)
        return self._make_position(node._left)

    def right(self, p):
        node = self._validate(p)
        return self._make_position(node._right)

    def num_children(self, p):
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1
        if node._right is not None:
            count += 1
        return count

    def _add_root(self, e):
        if self._root is not None:
            raise ValueError("Root exists")
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)

    def _add_left(self, p, e):
        node = self._validate(p)
        if node._left is not None:
            raise ValueError("Left child exists")
        self._size += 1
        node._left = self._Node(e, node)
        return self._make_position(node._left)

    def _add_right(self, p, e):
        node = self._validate(p)
        if node._right is not None:
            raise ValueError("Right child exists")
        self._size += 1
        node._right = self._Node(e, node)
        return self._make_position(node._right)

    def _replace(self, p, e):
        node = self._validate(p)
        old = node._element
        node._element = e
        return old

    def _delete(self, p):
        node = self._validate(p)
        if self.num_children(p) == 2:
            raise ValueError("Position has two children")
        child = node._left if node._left else node._right
        if child is not None:
            child._parent = node._parent
        if node is self._root:
            self._root = child
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node
        return node._element

    def _attach(self, p, t1, t2):
        node = self._validate(p)
        if not self.is_leaf(p):
            raise ValueError("position must be leaf")
        if not type(self) is type(t1) is type(t2):
            raise TypeError("Tree types must match")
        self._size += len(t1) + len(t2)
        if not t1.is_empty():
            t1._root._parent = node
            node._left = t1._root
            t1._root = None
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2._root
            t2._root = None
            t2._size = 0

    def compute_and_print_heights(self):
        def postorder_height(p):
            height = 0
            for c in self.children(p):
                height = max(height, postorder_height(c) + 1)
            print(f"Element: {p.element()}, Height: {height}")
            return height

        if not self.is_empty():
            postorder_height(self.root())


def list_to_tree(values):
    tree = LinkedBinaryTree()
    if not values:
        return tree

    iter_values = iter(values)
    root_value = next(iter_values)
    if root_value is not None:
        root = tree._add_root(root_value)
        queue = LinkedQueue()
        queue.enqueue(root)

        while not queue.is_empty():
            node = queue.dequeue()

            try:
                left_value = next(iter_values)
                if left_value is not None:
                    left_node = tree._add_left(node, left_value)
                    queue.enqueue(left_node)
            except StopIteration:
                break

            try:
                right_value = next(iter_values)
                if right_value is not None:
                    right_node = tree._add_right(node, right_value)
                    queue.enqueue(right_node)
            except StopIteration:
                break

    return tree


def draw_tree(tree, position=None, prefix="", is_tail=True):
    if position is None:
        position = tree.root()
    if position is not None:
        print(prefix + ("└── " if is_tail else "├── ") + str(position.element()))
        children = list(tree.children(position))
        for i, child in enumerate(children):
            draw_tree(
                tree,
                child,
                prefix + ("    " if is_tail else "│   "),
                i == len(children) - 1,
            )


# Create a binary tree from a list of values
tree = list_to_tree([1, 2, 3, 4, 5, 6, 7])
draw_tree(tree)
tree.compute_and_print_heights()


└── 1
    ├── 2
    │   ├── 4
    │   └── 5
    └── 3
        ├── 6
        └── 7
Element: 4, Height: 0
Element: 5, Height: 0
Element: 2, Height: 1
Element: 6, Height: 0
Element: 7, Height: 0
Element: 3, Height: 1
Element: 1, Height: 2


## Exercise 3

**If your first name starts with a letter from A-J inclusively:**

Give an alternative implementation of the HeapPriorityQueue’s upheap
method that uses recursion (and no loop). **Hint**: Do a single upward
swap and recur (if necessary).

In [13]:
class Empty(Exception):
    pass


class PriorityQueueBase:
    class _Item:
        __slots__ = "_key", "_value"

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __lt__(self, other):
            return self._key < other._key

        def __repr__(self):
            return "({0},{1})".format(self._key, self._value)

    def is_empty(self):
        return len(self) == 0

    def __len__(self):
        raise NotImplementedError("must be implemented by subclass")

    def add(self, key, value):
        raise NotImplementedError("must be implemented by subclass")

    def min(self):
        raise NotImplementedError("must be implemented by subclass")

    def remove_min(self):
        raise NotImplementedError("must be implemented by subclass")


class HeapPriorityQueue(PriorityQueueBase):
    def _parent(self, j):
        return (j - 1) // 2

    def _left(self, j):
        return 2 * j + 1

    def _right(self, j):
        return 2 * j + 2

    def _has_left(self, j):
        return self._left(j) < len(self._data)

    def _has_right(self, j):
        return self._right(j) < len(self._data)

    def _swap(self, i, j):
        self._data[i], self._data[j] = self._data[j], self._data[i]

    def _upheap(self, j):
        if j > 0:
            parent = self._parent(j)
            if self._data[j] < self._data[parent]:
                self._swap(j, parent)
                self._upheap(parent)

    def _downheap(self, j):
        if self._has_left(j):
            left = self._left(j)
            small_child = left
            if self._has_right(j):
                right = self._right(j)
                if self._data[right] < self._data[left]:
                    small_child = right
            if self._data[small_child] < self._data[j]:
                self._swap(j, small_child)
                self._downheap(small_child)

    def __init__(self):
        self._data = []

    def __len__(self):
        return len(self._data)

    def add(self, key, value):
        self._data.append(self._Item(key, value))
        self._upheap(len(self._data) - 1)

    def min(self):
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        item = self._data[0]
        return (item._key, item._value)

    def remove_min(self):
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        self._swap(0, len(self._data) - 1)
        item = self._data.pop()
        self._downheap(0)
        return (item._key, item._value)


def main():
    pq = HeapPriorityQueue()
    pq.add(5, 'A')
    pq.add(9, 'B')
    pq.add(3, 'C')
    pq.add(7, 'D')
    pq.add(2, 'E')

    print("Priority Queue elements (in heap order):")
    print(pq._data)

    print("\nMinimum element:")
    print(pq.min())

    print("\nRemoving minimum elements:")
    while not pq.is_empty():
        print(pq.remove_min())


if __name__ == "__main__":
    main()

Priority Queue elements (in heap order):
[(2,E), (3,C), (5,A), (9,B), (7,D)]

Minimum element:
(2, 'E')

Removing minimum elements:
(2, 'E')
(3, 'C')
(5, 'A')
(7, 'D')
(9, 'B')


In [11]:
class Empty(Exception):
    pass


class PriorityQueueBase:
    class _Item:
        __slots__ = "_key", "_value"

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __lt__(self, other):
            return self._key < other._key

        def __repr__(self):
            return "({0},{1})".format(self._key, self._value)

    def is_empty(self):
        return len(self) == 0

    def __len__(self):
        raise NotImplementedError("must be implemented by subclass")

    def add(self, key, value):
        raise NotImplementedError("must be implemented by subclass")

    def min(self):
        raise NotImplementedError("must be implemented by subclass")

    def remove_min(self):
        raise NotImplementedError("must be implemented by subclass")


class HeapPriorityQueue(PriorityQueueBase):
    def _parent(self, j):
        return (j - 1) // 2

    def _left(self, j):
        return 2 * j + 1

    def _right(self, j):
        return 2 * j + 2

    def _has_left(self, j):
        return self._left(j) < len(self._data)

    def _has_right(self, j):
        return self._right(j) < len(self._data)

    def _swap(self, i, j):
        self._data[i], self._data[j] = self._data[j], self._data[i]

    def _upheap(self, j):
        parent = self._parent(j)
        if j > 0 and self._data[j] < self._data[parent]:
            self._swap(j, parent)
            self._upheap(parent)

    def _downheap(self, j):
        if self._has_left(j):
            left = self._left(j)
            small_child = left
            if self._has_right(j):
                right = self._right(j)
                if self._data[right] < self._data[left]:
                    small_child = right
            if self._data[small_child] < self._data[j]:
                self._swap(j, small_child)
                self._downheap(small_child)

    def __init__(self):
        self._data = []

    def __len__(self):
        return len(self._data)

    def add(self, key, value):
        self._data.append(self._Item(key, value))
        self._upheap(len(self._data) - 1)

    def min(self):
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        item = self._data[0]
        return (item._key, item._value)

    def remove_min(self):
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        self._swap(0, len(self._data) - 1)
        item = self._data.pop()
        self._downheap(0)
        return (item._key, item._value)


def main():
    pq = HeapPriorityQueue()
    pq.add(5, 'A')
    pq.add(9, 'B')
    pq.add(3, 'C')
    pq.add(7, 'D')
    pq.add(2, 'E')

    print("Priority Queue elements (in heap order):")
    print(pq._data)

    print("\nMinimum element:")
    print(pq.min())

    print("\nRemoving minimum elements:")
    while not pq.is_empty():
        print(pq.remove_min())


if __name__ == "__main__":
    main()

Priority Queue elements (in heap order):
[(2,E), (3,C), (5,A), (9,B), (7,D)]

Minimum element:
(2, 'E')

Removing minimum elements:
(2, 'E')
(3, 'C')
(5, 'A')
(7, 'D')
(9, 'B')


**If your first name starts with a letter from K-Z inclusively:**

Reimplement the SortedPriorityQueue using java array. Make sure to
maintain removeMin’s O(1) performance.

(2 marks)

In [3]:
class Empty(Exception):
    pass


class PriorityQueueBase:
    class _Item:
        __slots__ = "_key", "_value"

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __lt__(self, other):
            return self._key < other._key

        def __repr__(self):
            return "({0},{1})".format(self._key, self._value)

    def is_empty(self):
        return len(self) == 0

    def __len__(self):
        raise NotImplementedError("must be implemented by subclass")

    def add(self, key, value):
        raise NotImplementedError("must be implemented by subclass")

    def min(self):
        raise NotImplementedError("must be implemented by subclass")

    def remove_min(self):
        raise NotImplementedError("must be implemented by subclass")


class SortedPriorityQueue(PriorityQueueBase):
    def __init__(self):
        self._data = []

    def __len__(self):
        return len(self._data)

    def add(self, key, value):
        newest = self._Item(key, value)
        # Insert in sorted order
        for i in range(len(self._data)):
            if newest < self._data[i]:
                self._data.insert(i, newest)
                break
        else:
            self._data.append(newest)

    def min(self):
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        item = self._data[0]
        return (item._key, item._value)

    def remove_min(self):
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        item = self._data.pop(0)
        return (item._key, item._value)


# Example usage
if __name__ == "__main__":
    pq = SortedPriorityQueue()
    pq.add(5, "A")
    pq.add(9, "C")
    pq.add(3, "B")
    pq.add(7, "D")

    print("Min:", pq.min())  # Should print (3, 'B')
    print("Remove Min:", pq.remove_min())  # Should remove and print (3, 'B')
    print("Min:", pq.min())  # Should print (5, 'A')

Min: (3, 'B')
Remove Min: (3, 'B')
Min: (5, 'A')


In [4]:
class Empty(Exception):
    """Error attempting to access an element from an empty container."""

    pass


class _DoublyLinkedBase:
    """A base class providing a doubly linked list representation."""

    # -------------------------- nested _Node class --------------------------
    # nested _Node class
    class _Node:
        """Lightweight, nonpublic class for storing a doubly linked node."""

        __slots__ = "_element", "_prev", "_next"  # streamline memory

        def __init__(self, element, prev, next):  # initialize node's fields
            self._element = element  # user's element
            self._prev = prev  # previous node reference
            self._next = next  # next node reference

    # -------------------------- list constructor --------------------------

    def __init__(self):
        """Create an empty list."""
        self._header = self._Node(None, None, None)
        self._trailer = self._Node(None, None, None)
        self._header._next = self._trailer  # trailer is after header
        self._trailer._prev = self._header  # header is before trailer
        self._size = 0  # number of elements

    # -------------------------- public accessors --------------------------

    def __len__(self):
        """Return the number of elements in the list."""
        return self._size

    def is_empty(self):
        """Return True if list is empty."""
        return self._size == 0

    # -------------------------- nonpublic utilities --------------------------

    def _insert_between(self, e, predecessor, successor):
        """Add element e between two existing nodes and return new node."""
        newest = self._Node(e, predecessor, successor)  # linked to neighbors
        predecessor._next = newest
        successor._prev = newest
        self._size += 1
        return newest

    def _delete_node(self, node):
        """Delete nonsentinel node from the list and return its element."""
        predecessor = node._prev
        successor = node._next
        predecessor._next = successor
        successor._prev = predecessor
        self._size -= 1
        element = node._element  # record deleted element
        node._prev = node._next = node._element = None  # deprecate node
        return element  # return deleted element


class PositionalList(_DoublyLinkedBase):
    """A sequential container of elements allowing positional access."""

    # -------------------------- nested Position class --------------------------
    class Position:
        """An abstraction representing the location of a single element.

        Note that two position instaces may represent the same inherent
        location in the list.  Therefore, users should always rely on
        syntax 'p == q' rather than 'p is q' when testing equivalence of
        positions.
        """

        def __init__(self, container, node):
            """Constructor should not be invoked by user."""
            self._container = container
            self._node = node

        def element(self):
            """Return the element stored at this Position."""
            return self._node._element

        def __eq__(self, other):
            """Return True if other is a Position representing the same location."""
            return type(other) is type(self) and other._node is self._node

        def __ne__(self, other):
            """Return True if other does not represent the same location."""
            return not (self == other)  # opposite of __eq__

    # ------------------------------- utility methods -------------------------------
    def _validate(self, p):
        """Return position's node, or raise appropriate error if invalid."""
        if not isinstance(p, self.Position):
            raise TypeError("p must be proper Position type")
        if p._container is not self:
            raise ValueError("p does not belong to this container")
        if p._node._next is None:  # convention for deprecated nodes
            raise ValueError("p is no longer valid")
        return p._node

    def _make_position(self, node):
        """Return Position instance for given node (or None if sentinel)."""
        if node is self._header or node is self._trailer:
            return None  # boundary violation
        else:
            return self.Position(self, node)  # legitimate position

    # ------------------------------- accessors -------------------------------
    def first(self):
        """Return the first Position in the list (or None if list is empty)."""
        return self._make_position(self._header._next)

    def last(self):
        """Return the last Position in the list (or None if list is empty)."""
        return self._make_position(self._trailer._prev)

    def before(self, p):
        """Return the Position just before Position p (or None if p is first)."""
        node = self._validate(p)
        return self._make_position(node._prev)

    def after(self, p):
        """Return the Position just after Position p (or None if p is last)."""
        node = self._validate(p)
        return self._make_position(node._next)

    def __iter__(self):
        """Generate a forward iteration of the elements of the list."""
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)

    # ------------------------------- mutators -------------------------------
    # override inherited version to return Position, rather than Node
    def _insert_between(self, e, predecessor, successor):
        """Add element between existing nodes and return new Position."""
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)

    def add_first(self, e):
        """Insert element e at the front of the list and return new Position."""
        return self._insert_between(e, self._header, self._header._next)

    def add_last(self, e):
        """Insert element e at the back of the list and return new Position."""
        return self._insert_between(e, self._trailer._prev, self._trailer)

    def add_before(self, p, e):
        """Insert element e into list before Position p and return new Position."""
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)

    def add_after(self, p, e):
        """Insert element e into list after Position p and return new Position."""
        original = self._validate(p)
        return self._insert_between(e, original, original._next)

    def delete(self, p):
        """Remove and return the element at Position p."""
        original = self._validate(p)
        return self._delete_node(original)  # inherited method returns element

    def replace(self, p, e):
        """Replace the element at Position p with e.

        Return the element formerly at Position p.
        """
        original = self._validate(p)
        old_value = original._element  # temporarily store old element
        original._element = e  # replace with new element
        return old_value  # return the old element value


class PriorityQueueBase:
    """Abstract base class for a priority queue."""

    # ------------------------------ nested _Item class ------------------------------
    class _Item:
        """Lightweight composite to store priority queue items."""

        __slots__ = "_key", "_value"

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __lt__(self, other):
            return self._key < other._key  # compare items based on their keys

        def __repr__(self):
            return "({0},{1})".format(self._key, self._value)

    # ------------------------------ public behaviors ------------------------------
    def is_empty(self):  # concrete method assuming abstract len
        """Return True if the priority queue is empty."""
        return len(self) == 0

    def __len__(self):
        """Return the number of items in the priority queue."""
        raise NotImplementedError("must be implemented by subclass")

    def add(self, key, value):
        """Add a key-value pair."""
        raise NotImplementedError("must be implemented by subclass")

    def min(self):
        """Return but do not remove (k,v) tuple with minimum key.

        Raise Empty exception if empty.
        """
        raise NotImplementedError("must be implemented by subclass")

    def remove_min(self):
        """Remove and return (k,v) tuple with minimum key.

        Raise Empty exception if empty.
        """
        raise NotImplementedError("must be implemented by subclass")


class SortedPriorityQueue(PriorityQueueBase):  # base class defines _Item
    """A min-oriented priority queue implemented with a sorted list."""

    # ------------------------------ public behaviors ------------------------------
    def __init__(self):
        """Create a new empty Priority Queue."""
        self._data = PositionalList()

    def __len__(self):
        """Return the number of items in the priority queue."""
        return len(self._data)

    def add(self, key, value):
        """Add a key-value pair."""
        newest = self._Item(key, value)  # make new item instance
        walk = self._data.last()  # walk backward looking for smaller key
        while walk is not None and newest < walk.element():
            walk = self._data.before(walk)
        if walk is None:
            self._data.add_first(newest)  # new key is smallest
        else:
            self._data.add_after(walk, newest)  # newest goes after walk

    def min(self):
        """Return but do not remove (k,v) tuple with minimum key.

        Raise Empty exception if empty.
        """
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        p = self._data.first()
        item = p.element()
        return (item._key, item._value)

    def remove_min(self):
        """Remove and return (k,v) tuple with minimum key.

        Raise Empty exception if empty.
        """
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        item = self._data.delete(self._data.first())
        return (item._key, item._value)


# Example usage
if __name__ == "__main__":
    pq = SortedPriorityQueue()
    pq.add(5, "A")
    pq.add(9, "C")
    pq.add(3, "B")
    pq.add(7, "D")

    print("Min:", pq.min())  # Should print (3, 'B')
    print("Remove Min:", pq.remove_min())  # Should remove and print (3, 'B')
    print("Min:", pq.min())  # Should print (5, 'A')

Min: (3, 'B')
Remove Min: (3, 'B')
Min: (5, 'A')
